<a href="https://colab.research.google.com/github/Mayaralmeid23/guia-da-bolsa/blob/main/Guia_de_bolsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Guia da Bolsa 💼  
Um assistente com IA para apoiar investidores iniciantes a entender, decidir e pensar com clareza antes de investir.

---

## ✨ O que este projeto faz

Este notebook utiliza inteligência artificial generativa (Gemini) e dados reais do mercado para ajudar o usuário a:

- 📊 Buscar informações sobre ações ou ETFs
- 🧾 Entender termos financeiros com linguagem acessível
- 🧠 Refletir com calma antes de tomar decisões de investimento

O sistema é dividido em três agentes especializados:
1. **Decisão** – ajuda a analisar ativos e fazer escolhas
2. **Instrução** – explica termos e conceitos de forma simples
3. **Clareza** – apoia emocionalmente quem está inseguro, com orientações leves e sem julgamento

---

## 🧭 Como usar este notebook

1. Adicione sua chave da API Gemini no painel lateral como `GOOGLE_API_KEY`
2. Execute todas as células, de cima para baixo
3. No final, interaja com o assistente digitando mensagens
4. Para sair da conversa, digite `sair`

---

## 🧠 Sobre a personalidade da IA

Você está interagindo com uma assistente empática, profissional e gentil.  
Ela fala com clareza, sem jargões técnicos, e orienta com respeito ao tempo e ao nível do usuário.

Ela não está aqui apenas para responder — mas para acolher, encorajar e ajudar a tomar boas decisões com segurança.

---

## 🧪 Tecnologias usadas

- **Python** + Google Colab
- **Gemini API (google.generativeai)**
- **Yahoo Finance (via `yfinance`)**
- **Engenharia de Prompt**
- **Memória curta com histórico de conversa**

---

Vamos começar?
O que você gostaria de saber ou refletir hoje? 💬



In [131]:
# Limpar histórico de conversa (reset manual)
historico_conversa = []


In [75]:
# Instala a biblioteca do Gemini
!pip install -q -U google-generativeai


In [76]:
from google.colab import userdata
API_KEY = userdata.get("GOOGLE_API_KEY")

import google.generativeai as genai
genai.configure(api_key=API_KEY)

modelo_gemini = genai.GenerativeModel(model_name="gemini-1.5-flash")


In [77]:
!pip install yfinance

In [78]:
import yfinance as yf

def buscar_acao_yahoo(ticker):
    try:
        acao = yf.Ticker(ticker.upper() + ".SA")
        dados = acao.history(period="1d")
        info = acao.info

        preco = round(dados["Close"].iloc[-1], 2)
        variacao = round(dados["Close"].iloc[-1] - dados["Open"].iloc[-1], 2)
        nome_empresa = info.get("longName", "Empresa não identificada")
        setor = info.get("sector", "Setor não informado")
        dividend_yield = round(info.get("dividendYield", 0) * 100, 2)

        resposta = f"""
📊 {ticker.upper()} ({nome_empresa})
Setor: {setor}
Preço atual: R$ {preco}
Variação do dia: R$ {variacao:+}
Dividend yield: {dividend_yield}%

Posso te ajudar a interpretar esses dados, se quiser.
"""
        return resposta.strip()

    except Exception as e:
        return "Não consegui encontrar esse ativo agora. Tente com outro código como PETR4, VALE3 ou ITUB4."


In [132]:
historico_conversa = []


In [80]:
def gerar_prompt_com_memoria(mensagem_usuario):
    historico_conversa.append(f"Usuário: {mensagem_usuario}")

    if len(historico_conversa) > 6:
        historico_conversa.pop(0)

    contexto = "\n".join(historico_conversa)

    prompt = f"""
Você é uma assistente de investimentos para iniciantes.
Seu papel é ser **direta, acolhedora e confiável**.

Responda sempre com:
- Clareza e sem metáforas
- Linguagem simples e objetiva
- Explicações acessíveis, com exemplos reais quando necessário
- Uma pergunta relevante ao final, para incentivar a continuidade

Abaixo está o histórico recente da conversa.
Use esse contexto para responder com empatia e precisão:

{contexto}

IA:
"""
    return prompt



In [141]:
import re

def agente_decisao(input_usuario):
    historico_conversa.append(f"Usuário: {input_usuario}")
    if len(historico_conversa) > 6:
        historico_conversa.pop(0)

    # Detectar códigos como PETR4, VALE3, IVVB11 etc.
    ativos_mencionados = re.findall(r'\b[A-Z]{4}\d{1,2}\b', input_usuario.upper())

    if ativos_mencionados:
        respostas = [buscar_acao_yahoo(ativo) for ativo in ativos_mencionados]
        return "\n".join(respostas)

    # Prompt personalizado para decisões sem código direto
    contexto = "\n".join(historico_conversa)

    prompt = f"""
Você é uma assistente de investimentos especializada em **renda variável**, com foco em apoiar iniciantes na tomada de decisões seguras e informadas.

Sua missão é:
- Analisar dúvidas e objetivos do usuário
- Sugerir ativos compatíveis com seu perfil, como ações, ETFs, REITs, BDRs ou setores específicos
- Focar em critérios reais: volatilidade, liquidez, dividendos, histórico
- Evitar recomendações genéricas ou produtos de renda fixa
- Sempre encerrar com uma pergunta estratégica que ajude o usuário a evoluir na decisão

Responda com:
- Frases diretas e empáticas
- Linguagem simples, sem metáforas nem jargões
- Uma ideia por vez
- Tom encorajador e gentil
- Nunca diga “invista em” ou “você deve”
- Prefira: “você pode explorar”, “uma alternativa pode ser”, “algumas pessoas iniciam com”

Histórico recente da conversa:

{contexto}

IA:
"""
    resposta = modelo_gemini.generate_content(prompt)
    historico_conversa.append(f"IA: {resposta.text.strip()}")
    return resposta.text.strip()


In [142]:
def agente_instrucao(input_usuario):
    historico_conversa.append(f"Usuário: {input_usuario}")
    if len(historico_conversa) > 6:
        historico_conversa.pop(0)
    contexto = "\n".join(historico_conversa)

    prompt = f"""
Você é uma especialista em investimentos com foco em **renda variável**, orientando iniciantes com clareza e assertividade.

Sua missão é:
- Explicar termos relacionados ao mercado de ações, BDRs, ETFs, dividendos, análises técnicas e fundamentos
- Evitar jargões técnicos sem explicação
- Manter a linguagem acessível, mas sem infantilização
- Utilizar exemplos realistas quando fizer sentido
- Terminar com uma pergunta direta para manter a conversa fluindo

Sempre responda com:
Use frases diretas e empáticas. Evite termos técnicos sem explicação simples. Fale como se estivesse conversando com alguém que está aprendendo do zero. Quando citar um conceito, explique com palavras do dia a dia, sem metáforas nem jargões. Traga apenas uma ideia por vez. Seja gentil e encorajador. Nunca recomende diretamente um ativo ou produto específico. Em vez disso, ajude o usuário a entender suas opções com base no que ele busca, e estimule a reflexão. Evite frases como “invista em” ou “você deve”. Prefira: “você pode explorar”, “uma alternativa pode ser”, “algumas pessoas iniciam com”.
Histórico recente da conversa:

{contexto}

IA:
"""
    resposta = modelo_gemini.generate_content(prompt)
    historico_conversa.append(f"IA: {resposta.text.strip()}")
    return resposta.text.strip()


In [143]:
def agente_clareza(input_usuario):
    historico_conversa.append(f"Usuário: {input_usuario}")
    if len(historico_conversa) > 6:
        historico_conversa.pop(0)
    contexto = "\n".join(historico_conversa)

    prompt = f"""
Você é uma assistente empática e profissional que ajuda pessoas iniciantes a começarem a investir em **renda variável**, mesmo com pouco dinheiro ou conhecimento prévio.

Sua missão é:
- Validar o sentimento do usuário com respeito e acolhimento
- Focar em investimentos de renda variável: ações, ETFs, dividendos, BDRs
- Reforçar que é possível começar com pouco, com segurança e estratégia
- Evitar metáforas infantis ou analogias exageradas
- Encerrar com uma pergunta que estimule a clareza e continuidade da conversa

Sempre responda com:
Use frases diretas e empáticas. Evite termos técnicos sem explicação simples. Fale como se estivesse conversando com alguém que está aprendendo do zero. Quando citar um conceito, explique com palavras do dia a dia, sem metáforas nem jargões. Traga apenas uma ideia por vez. Seja gentil e encorajador. Nunca recomende diretamente um ativo ou produto específico. Em vez disso, ajude o usuário a entender suas opções com base no que ele busca, e estimule a reflexão. Evite frases como “invista em” ou “você deve”. Prefira: “você pode explorar”, “uma alternativa pode ser”, “algumas pessoas iniciam com”.

Histórico recente da conversa:

{contexto}

IA:
"""
    resposta = modelo_gemini.generate_content(prompt)
    historico_conversa.append(f"IA: {resposta.text.strip()}")
    return resposta.text.strip()


In [130]:
def roteador_agente(input_usuario):
    historico_conversa.append(f"Usuário: {input_usuario}")
    if len(historico_conversa) > 6:
        historico_conversa.pop(0)

    texto = input_usuario.lower()

    # 👇 Atalhos para identificar intenção com base em palavras-chave
    if any(p in texto for p in ["preço", "valor", "quanto está", "cotação", "subiu", "caiu", "vale", "quanto custa", "quanto tá"]):
        return agente_decisao(input_usuario)

    if any(p in texto for p in ["explica", "o que é", "significa", "termo", "contrato", "jargão"]):
        return agente_instrucao(input_usuario)

    if any(p in texto for p in ["medo", "ansioso", "dúvida", "receio", "não sei o que fazer", "insegurança"]):
        return agente_clareza(input_usuario)

    # Fallback inteligente com Gemini como roteador
    contexto = "\n".join(historico_conversa)

    prompt = f"""
Você é o roteador de um assistente financeiro com 3 agentes especialistas:

1. agente_instrucao: explica termos e jargões
2. agente_decisao: ajuda a escolher ativos ou tomar decisões de investimento
3. agente_clareza: acolhe dúvidas emocionais e inseguranças

Sua função é analisar a última frase do usuário e escolher **um único agente apropriado**.
Retorne apenas o nome do agente ideal: `agente_instrucao`, `agente_decisao` ou `agente_clareza`.

Histórico da conversa:
{contexto}

Com base na última mensagem, qual agente deve responder?
"""
    resposta = modelo_gemini.generate_content(prompt)
    agente_escolhido = resposta.text.strip().lower()

    if "instrucao" in agente_escolhido:
        return agente_instrucao(input_usuario)
    elif "decisao" in agente_escolhido:
        return agente_decisao(input_usuario)
    elif "clareza" in agente_escolhido:
        return agente_clareza(input_usuario)
    else:
        return "Desculpe, não consegui entender sua pergunta. Você pode tentar reformular com mais detalhes?"


In [ ]:

# Mensagem inicial automática (fora da função roteador)
print("Olá! Eu sou o Guia da Bolsa 🧠💼")
print("Estou aqui para te ajudar a investir com mais clareza e confiança.")
print("Você pode me pedir para:\n"
      "1 - Explicar um termo ou jargão\n"
      "2 - Ver dados de uma ação (ex: PETR4, VALE3)\n"
      "3 - Refletir antes de tomar uma decisão\n")

while True:
    pergunta = input("\nDigite sua pergunta ou número da opção (ou escreva 'sair' para encerrar): ").strip()

    if pergunta.lower() in ["sair", "encerrar", "obrigado", "obrigada"]:
        print("Tudo bem! Estarei por aqui se precisar de mim de novo.")
        break

    resposta = roteador_agente(pergunta)
    print("\n" + resposta)


Olá! Eu sou o Guia da Bolsa 🧠💼
Estou aqui para te ajudar a investir com mais clareza e confiança.
Você pode me pedir para:
1 - Explicar um termo ou jargão
2 - Ver dados de uma ação (ex: PETR4, VALE3)
3 - Refletir antes de tomar uma decisão


Digite sua pergunta ou número da opção (ou escreva 'sair' para encerrar): Tenho medo de investir

Entendo seu medo.  Investir em renda variável, de fato, envolve riscos. Mas é importante lembrar que começar com pouco dinheiro e focar em aprender gradualmente pode minimizar esses riscos.  Um bom começo é entender que, ao investir em ETFs, você está diversificando, ou seja, colocando seu dinheiro em várias empresas diferentes, ao invés de apenas uma.  Isso reduz o impacto caso alguma delas não vá bem.  Vamos começar devagar, ok?  Quer conversar mais sobre como a diversificação ajuda a controlar o risco?

Digite sua pergunta ou número da opção (ou escreva 'sair' para encerrar): Qual preço da PETR4?

📊 PETR4 (Petróleo Brasileiro S.A. - Petrobras)
Setor